используем модель, второй раз парсим, получаем товары с сайта

In [11]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import requests
from selenium import webdriver
import time
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
import pickle

In [18]:
opros = pd.read_csv("final_opros.csv") 
opros = opros.drop(columns = "Unnamed: 0")
cols = list(opros.columns)
X = np.array(opros[cols[:5] + cols[-12:]])
Y = np.array(opros[cols[6:-12]])

In [20]:
regr = pickle.load(open("finalized_model.sav", 'rb'))
regr.predict([X[0]]) #проверка

array([[7.42857143, 6.        , 7.84615385, 9.125     , 9.11111111,
        5.11111111, 9.68421053, 3.375     , 5.68421053, 3.54285714,
        9.33333333, 6.94117647, 8.15384615, 1.14285714, 0.5625    ,
        4.42105263, 0.46153846, 1.        , 1.5       , 3.52941176,
        3.42857143, 6.10526316, 6.36363636, 5.73333333, 7.46666667,
        3.5       , 7.38095238, 7.09677419, 6.41176471, 6.56521739,
        5.47058824, 5.23076923, 3.21428571, 0.54545455, 2.09090909,
        7.36842105, 8.08695652, 2.88888889, 5.72727273, 4.64      ,
        5.63636364, 8.42105263, 7.83333333, 6.44897959, 8.23529412,
        4.05405405, 4.75862069]])

In [26]:
chosen_categories = np.array(['Смартфоны и умные часы',
 'Портативная техника',
 'Аудио- и видеотехника',
 'Всё для геймеров',
 'Фото- и видеокамеры',
 'Робототехника и Stem-игрушки',
 'Игры и приставки',
 'GPS-навигация',
 'Техника для дома',
 'Техника для красоты',
 'Ноутбуки и настольные ПК',
 'Аксессуары',
 'Игрушки и игры',
 'Коляски и автокресла',
 'Масла и технические жидкости',
 'Аксессуары и оборудование',
 'Автомобильные инструменты',
 'Противоугонные устройства',
 'Мототехника',
 'Мебель',
 'Велоспорт',
 'Самокаты',
 'Моноколеса и гироскутеры',
 'Скейтбординг',
 'Роликовые коньки',
 'Спортивная защита',
 'Туризм и отдых на природе',
 'Тренажеры и фитнес',
 'Уход за волосами',
 'Уход за лицом',
 'Уход за телом',
 'Парфюмерия',
 'Оптика',
 'Ортопедические изделия',
 'Массажеры',
 'Чай, кофе, какао',
 'Кондитерские изделия',
 'Здоровое питание',
 'Все для выпечки',
 'Алкоголь',
 'Сумки и чемоданы ',
 'Музыкальные инструменты',
 'Книги',
 'Игры для компаний',
 'Билеты',
 'Гадания и предсказания',
 'Квадрокоптеры'])

In [28]:
# пусть это прибыло
person = X[55]
min_price = 200
max_price = 2000

In [22]:
def predict_for_person(person):
    if (type(person) != type(np.zeros(1)) or len(person)!= 17):
        print("ошибка, человек описан неверно")
    else:
        return regr.predict([person])[0]

In [30]:
def make_some_gift(person, min_price, max_price):
    cats = predict_for_person(person)
    cats = np.argsort(cats)[::-1]
    print(chosen_categories[cats])
    '''
    Вот в этом месте мы идём на сайт, вбиваем в поиск название категории, парсим результаты и смотрим, есть ли в первых 10-ти 
    результатах то, что укладывается по цене. Если есть - выводим и идем к следующей категории. Так выкладываем
    три товара. При нажатии кнопки "ещё" идем дальше по категориям. Если доходим до конца пишем "это всё"
    '''